# Layered counting windows

My first approach to this competition used a collection of strategies and kept a count of how frequently each would win, selecting the strategy with the maximum win count.  The first problem anyone using this approach encounters is that once a strategy accumulates a high win count, the opponent might start to target that strategy, and it will take some time before the win count declines enough to reorient the agent properly.

One approach (not my strongest approach, but a strong one) was to use multiple rolling windows for counting.  In the code, I refer to "weight schemes," which are collections of rolling windows that use a weighted average to return a score.  Since we can think of a "strategy" as an action-selection and a record of hits and misses, we can use weight schemes to convert a collection of strategies into a new collection of strategies (the diagram below is intended to show how).

When we layer this technique upon itself, the agent seems to be better equipped to evade detection, and to dynamically target different unique opponent strategies. But it is also prone to more random fluctuation, which  can lead to sporadically losing matches. 

Today, near the end of this competition, the strongest variation of this strategy sits at 1016, or about 14th place (silver).

(Write-up and public notebook for my strongest agent are coming soon.  Since this is quite different from my strongest, I figured I'd make it public as well.)

![diagram](https://i.imgur.com/MMLu3zN.png)

In [ ]:
%%writefile submission.py

import os
import secrets

NUM_SCHEMES = 6
NUM_SCHEMES_FOLLOWUP = 6 
NUM_SCHEMES_FINAL = 4

my_last_action = None
token_list = []
tally = 0
log = ''
act_highest = -1
act_total = -1
act_highest_record = []
act_total_record = []
beats = [1, 2, 0]
frame = 0
records = []
wins = []
last_guesses = []

def randomPlay():
    return secrets.randbelow(3)

#####
# Counts values in a series with a rolling window of some length, returning a 
# distribution of value-counts from that period
#####
class RollingCounter:
    def __init__(self, length):
        self.length = length
        self.items = []
        self.totals = [0, 0, 0]
        self.currIndex = 0
        self.cutoffIndex = self.length * -1

    def count(self, val):
        self.items.append(val)
        self.totals[val] += 1
        if self.cutoffIndex > 0:
            self.totals[self.items[self.cutoffIndex]] -= 1
        self.cutoffIndex += 1

    def getDist(self):
        return self.totals

#####
# Counts values in a series with several rolling windows, returning a weighted 
# sum of these distributions
#####
class GangCounter:
    def __init__(self, lengthList):
        self.lengthList = lengthList
        self.lengthDict = {}
        self.counters = []
        for i, length in enumerate(lengthList):
            counter = RollingCounter(length)
            self.counters.append(counter)
            self.lengthDict[length] = counter

    def setWeightList(self, weights):
        self.weightList = weights

    def count(self, val):
        for counter in self.counters:
            counter.count(val)

    def getDist(self, length):
        counter = self.counters[self.lengthList.index(length)]
        return counter.getDist()

    def getDists(self):
        allDists = []
        for length in self.lengthList:
            dist = self.getDist(length)
            allDists.append(dist)
        return allDists

    def getWeightedVal(self):
        outputVal = 0.0
        totalWeight = 0
        for i, length in enumerate(self.lengthList):
            dist = self.getDist(length)
            if not dist[0] + dist[1] == 0:
                outputVal += (dist[1] / (dist[0] + dist[1])) * self.weightList[i]
                totalWeight += self.weightList[i]
        if totalWeight == 0:
            return 0
        return outputVal / totalWeight

#####
# Holds a collection of GangCounters that each return a weighted distribution 
# of value-counts from a series.
#####
class CompoundGangCounter:
    def __init__(self):
        self.counters = []

    def addWeights(self, lengthList, weightList):
        counter = GangCounter(lengthList)
        counter.setWeightList(weightList)
        self.counters.append(counter)

        
    ####
    # GangCounter weights used for evaluating the records of strategies
    ####
    def setDefaultWeights(self):
        self.addWeights([10, 7],[1, 1])
        self.addWeights([125],[1])
        self.addWeights([225],[1])
        self.addWeights([300, 50],[1, 1])
        self.addWeights([50, 20],[1, 1])
        self.addWeights([5,4,3,2,1],[10,11,12,13,14])


    ####
    # Weights for evaluating the next layer after strategy records are fed
    # through a FollowupTree class
    ####
    def setDefaultWeightsFollowup(self):
        self.addWeights([20, 10],[1,1])
        self.addWeights([125],[1])
        self.addWeights([7,3],[1, 1])
        self.addWeights([500, 50],[1, 1])
        self.addWeights([50, 5],[1, 1])
        self.addWeights([5,2,1],[1,1,1])


    ####
    # Weights for evaluating the final layer, which compares weighted results
    # from strategy records and the results of running those records though
    # FollowupTrees
    ####
    def setFinalWeights(self):
        self.addWeights([900],[1])
        self.addWeights([10, 7],[1, 1])
        self.addWeights([500,50],[1,1])
        self.addWeights([5,4,3, 2, 1], [1,1,1,1,1])


    def count(self, val):
        for counter in self.counters:
            counter.count(val)

    def getWeightedVals(self):
        vals = []
        for weightListIndex in range(len(self.counters)):
            vals.append(self.counters[weightListIndex].getWeightedVal())
        return vals

#####
# An ngram tree that lets us access a list and distribution of next plays 
# for each player after some n-gram of tokens, where a token is a unique 
# combination of this agent's play and the opponent agent's play.
#####
class PatternTree:
    def __init__(self):
        self.val = None
        self.root = True
        self.count = 0
        self.indexList = []
        self.myNextPlays = []
        self.oppNextPlays = []
        self.myNextPlaysDist = [0,0,0]
        self.oppNextPlaysDist = [0,0,0]
        self.children = {}

    def addNextPlays(self, myNext, oppNext):
        if not myNext == 'X':
            self.myNextPlays.append(myNext)
            self.myNextPlaysDist[int(myNext)] += 1
        if not oppNext == 'X':
            self.oppNextPlays.append(oppNext)
            self.oppNextPlaysDist[int(oppNext)] += 1

    def setVal(self, val, index):
        self.val = val
        self.count += 1
        self.indexList.append(index)

    def addIndex(self, ind):
        self.count += 1
        self.indexList.append(ind)

    def addSequence(self, sequence, maxAddLen):
        for i in range(maxAddLen):
            self.countPattern(sequence[len(sequence)-i:len(sequence)], 0)

    def countPattern(self, pattern):
        return self.countPattern(pattern, 0)

    def countPattern(self, pattern, index):
        if len(pattern) > 0:
            first = pattern[0]
            nextToken = 'XX'
            if len(pattern) > 1:
                nextToken = pattern[1]
            oppNext = nextToken[0]
            myNext = nextToken[1]
            rest = pattern[1:len(pattern)]
            child = self.children.get(first, None)
            if child is None:
                self.children[first] = PatternTree()
                self.children[first].setVal(first, index)
            else:
                self.children[first].addIndex(index)
            self.children[first].addNextPlays(myNext, oppNext)
            self.children[first].countPattern(rest, index + 1)

    def findPatternEndPoint(self, pattern):
        if len(pattern) > 0:
            first = pattern[0]
            rest = pattern[1:len(pattern)]
            child = self.children.get(first, None)
            if child is not None:
                return self.children[first].findPatternEndPoint(rest)
            else:
                return None
        else:
            return self

#####
# A PatternTree variant designed to count zeros and ones and that counts 
# results with a CompoundGangCounter.  The records of our strategies are run 
# through FollowupTrees to help detect complex patterns in how our opponent 
# chooses their next action.
#####
class FollowupTree:
    def __init__(self):
        self.val = None
        self.count = 0
        self.nextCountZeros = 0
        self.nextCountOnes = 0
        self.nextCount = CompoundGangCounter()
        self.nextCount.setDefaultWeightsFollowup()
        self.children = [None, None, None]

    def getWinProb(self):
        if float(self.nextCountOnes) + float(self.nextCountZeros) < 2:
            return 0
        else:
            return float(self.nextCountOnes) / (float(self.nextCountOnes) + float(self.nextCountZeros))

    def addNext(self, nextElement):
        if nextElement == 0:
            self.nextCountZeros += 1
            self.nextCount.count(0)

        elif nextElement == 1:
            self.nextCountOnes += 1
            self.nextCount.count(1)

    def getWeightedVals(self):
        return self.nextCount.getWeightedVals()

    def setVal(self, val):
        self.val = val
        self.count += 1

    def addSequence(self, sequence, maxAddLen):
        for i in range(maxAddLen):
            self.countPattern(sequence[len(sequence)-i:len(sequence)])

    def countPattern(self, pattern):
        if len(pattern) > 0:
            self.count += 1
            first = pattern[0]
            nextToken = 2
            if len(pattern) > 1:
                nextToken = pattern[1]

            rest = pattern[1:len(pattern)]
            child = self.children[first]
            if child is None:
                self.children[first] = FollowupTree()
                self.children[first].setVal(first)
            self.children[first].addNext(nextToken)
            self.children[first].countPattern(rest)

    def findPatternEndPoint(self, pattern):
        if len(pattern) > 0:
            first = pattern[0]
            rest = pattern[1:len(pattern)]
            child = self.children[first]
            if child is not None:
                return self.children[first].findPatternEndPoint(rest)
            else:
                return None
        else:
            return self

#####
# Update our pattern tree each frame
#####
patternTree = PatternTree()
def update_ngram_tree(tokenList):
    global patternTree
    patternTree.addSequence(tokenList, 15)
    return patternTree

def get_ngram(ngramLength, tokenList):
    return tokenList[len(tokenList) - ngramLength:len(tokenList)]

###############
# NGRAM STRATEGIES
#
# These are strategies that are analyzed each step for each of various ngram lengths. 
# We are matching patterns of multiple lengths from earlier in the match.  Each of these 
# strategies will be analyzed with two rotational "counter-strategies"
###############

#####
# Strategy based on what we played last after this ngram
#####
def strategy_changeup(tree, ngramLength, tokenList):
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None or len(treeNode.myNextPlays) == 0:
        return randomPlay()
    myLastPlay = treeNode.myNextPlays[len(treeNode.myNextPlays) - 1]
    return beats[int(myLastPlay)]

#####
# Strategy based on what the opponent played last after this ngram
#####
def strategy_opp_repeat(tree, ngramLength, tokenList):
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None or len(treeNode.myNextPlays) == 0:
        return randomPlay()
    oppLastPlay = treeNode.oppNextPlays[len(treeNode.oppNextPlays) - 1]
    return int(oppLastPlay)

#####
# Strategy that alternates what we play based on what we played last
#####
def strategy_alternate_based_on_my_last(tree, ngramLength, tokenList):
    global frame
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None or len(treeNode.myNextPlays) == 0:
        return randomPlay()
    myLastPlay = treeNode.myNextPlays[len(treeNode.myNextPlays) - 1]
    if frame % 2 == 0:
        return beats[int(myLastPlay)]
    else:
        return beats[beats[int(myLastPlay)]]

#####
# Strategy that alternates what we play based on what the opponent played last
#####
def strategy_alternate_based_on_opp_last(tree, ngramLength, tokenList):
    global frame
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None or len(treeNode.myNextPlays) == 0:
        return randomPlay()
    oppLastPlay = treeNode.oppNextPlays[len(treeNode.oppNextPlays) - 1]
    if frame % 2 == 0:
        return beats[int(oppLastPlay)]
    else:
        return beats[beats[int(oppLastPlay)]]

#####
# Strategy that expects the opponent to play after this ngram what they 
# typically play after this ngram
#####
def strategy_predict_by_frequency(tree, ngramLength, tokenList):
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None:
        return randomPlay()
    dist = treeNode.oppNextPlaysDist
    for i in [0, 1, 2]:
        if dist[i] > dist[(i + 1) % 3] and dist[i] > dist[(i + 2) % 3]:
            return beats[i]
    return randomPlay()

#####
# Strategy to play whatever we play least after this ngram
#####
def strategy_my_lowest_freq(tree, ngramLength, tokenlist):
    ngram = get_ngram(ngramLength, token_list)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None:
        return randomPlay()
    dist = treeNode.myNextPlaysDist
    for i in [0, 1, 2]:
        if dist[i] < dist[(i + 1) % 3] and dist[i] < dist[(i + 2) % 3]:
            return beats[i]
    return randomPlay()

#####
# Strategy to counter an agent that expects our most frequent action
#####
def strategy_my_highest_freq(tree, ngramLength, tokenList):
    ngram = get_ngram(ngramLength, token_list)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None:
        return randomPlay()
    dist = treeNode.myNextPlaysDist
    return beats[beats[dist.index(max(dist))]]

#####
# Strategy to counter an agent that always plays its own least frequent action
#####
def strategy_opp_lowest_freq(tree, ngramLength, tokenList):
    ngram = get_ngram(ngramLength, token_list)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None:
        return randomPlay()
    dist = treeNode.oppNextPlaysDist
    return beats[dist.index(min(dist))]

def make_distribution(list):
    dist = [0.0, 0.0, 0.0]
    for li in list:
        dist[int(li)] += 1

    return dist

#####
# Strategy that predicts the opponent will play what they played frequently in 
# the last 4 steps
#####
def strategy_predict_by_frequency_last_four(tree, ngramLength, tokenList):
    n = 4
    ngram = get_ngram(ngramLength, tokenList)
    treeNode = tree.findPatternEndPoint(ngram)
    if treeNode is None:
        return randomPlay()
    dist = make_distribution(treeNode.oppNextPlays[len(treeNode.oppNextPlays) - n:len(treeNode.oppNextPlays)])
    for i in [0, 1, 2]:
        if dist[i] > dist[(i + 1) % 3] and dist[i] > dist[(i + 2) % 3]:
            return beats[i]
    return randomPlay()
 

#####
# Evaluates a strategy and its historical record and returns the strategy's 
# recommendation and the record of that strategy as measured by a CompoundGangCounter,
# also uses Followup Trees to analyze this strategy's record.
#####
def evaluate_strategy(strategyList, strategyIndex, records, wins, last_guesses, tree, tokenList, ngramLength, counter):
    global followup_trees
    global NUM_SCHEMES
    strategy = strategyList[strategyIndex]
    suggestion = strategy(tree, ngramLength, tokenList)
    if counter >= 1:
        suggestion = beats[suggestion]
    if counter >= 2:
        suggestion = beats[suggestion]
    opponentLastPlay = int(tokenList[len(tokenList) - 1][0])
    if last_guesses[counter][strategyIndex][ngramLength][0] == beats[opponentLastPlay]:
        wins[counter][strategyIndex][ngramLength].count(1)
        records[counter][strategyIndex][ngramLength][0].append(1)
    else:
        wins[counter][strategyIndex][ngramLength].count(0)
        records[counter][strategyIndex][ngramLength][0].append(0)

    last_guesses[counter][strategyIndex][ngramLength][0] = suggestion
    if len(records[counter][strategyIndex][ngramLength][0]) > 3:
        scores = wins[counter][strategyIndex][ngramLength].getWeightedVals()
    else:
        scores = []
        for i in range(NUM_SCHEMES):
            scores.append(0)

    followupScores = search_followup_tree(records[counter][strategyIndex][ngramLength][0], followup_trees[counter][strategyIndex][ngramLength])
    return suggestion, scores, followupScores

#####
# Uses FollowupTree class to search for ngram patterns in a strategy's record itself
#####
def search_followup_tree(records, tree):
    global NUM_SCHEMES
    maxLength = 5
    lenAdd = 6
    for i in range(max(len(records) - lenAdd, 0)):
        tree.countPattern(records[(len(records) - lenAdd) + i:len(records)])

    highestWeightedVals = []
    for i in range(NUM_SCHEMES):
        highestWeightedVals.append(0)
    for ngramLen in range(maxLength):
        ngram = get_ngram(ngramLen, records)
        treeNode = tree.findPatternEndPoint(ngram)
        if treeNode is not None:
            if ngramLen > 0:
                weightedVals = treeNode.getWeightedVals()
                for i, weightedVal in enumerate(weightedVals):
                    highestWeightedVals[i] = max(weightedVal, highestWeightedVals[i])
    return highestWeightedVals

followup_trees = []
my_last_actions = []
my_last_actions_followup = []
score_records = []
score_records_followup = []
for i in range(NUM_SCHEMES):
    my_last_actions.append(0)
    my_last_actions_followup.append(0)
    score_records.append(CompoundGangCounter())
    score_records[len(score_records) - 1].setFinalWeights()
    score_records_followup.append(CompoundGangCounter())
    score_records_followup[len(score_records_followup) - 1].setFinalWeights()
    
final_records = []
final_records_followup = []
final_actions = []
final_actions_followup = []
final_records_counter = []
final_records_counter_followup = []
    
for j in range(NUM_SCHEMES_FINAL):
    final_records.append(0)
    final_records_counter.append(GangCounter([225]))
    final_records_counter[j].setWeightList([1])
    final_records_counter_followup.append(GangCounter([225]))
    final_records_counter_followup[j].setWeightList([1])
    final_actions.append(0)
    final_actions_followup.append(0)
    final_records_followup.append(0)
    
    
#####
# The driver of this agent, gets a matrix of scores from the compound gang counters 
# attached to the record of each ngram strategy, and a second matrix of scores from 
# the followup trees attached to each record, then evaluates the record of each 
# GangCounter's max-strategy record for each matrix and treats the output as a new 
# collection of records.
#
# Finally, we evaluate these last records using the "final" GangCounter weight 
# scheme: [225]
#####
def evaluate_records(observation, configuration, includeNew):
    global NUM_SCHEMES
    strategyList = [
        strategy_predict_by_frequency,
        strategy_changeup,
        strategy_my_lowest_freq,
        strategy_predict_by_frequency_last_four,
        strategy_alternate_based_on_my_last,
        strategy_alternate_based_on_opp_last,
        strategy_opp_repeat,
        strategy_opp_lowest_freq,
        strategy_my_highest_freq
    ]
    global followup_trees
    global records, wins, last_guesses, token_list, log, frame
    global my_last_actions, my_last_actions_followup, final_actions, final_actions_followup, score_records
    global final_records_counter, final_records_counter_followup, final_records_followup
    global match_records
    ngramSizes = 9
    
    if len(records) == 0 or records is None:
        records = []
        wins = []
        last_guesses = []
        followup_trees = []
        for counter_num in [0, 1, 2]:
            records.append([])
            wins.append([])
            last_guesses.append([])
            followup_trees.append([])
            for strategy_num in range(len(strategyList)):
                records[counter_num].append([])
                wins[counter_num].append([])
                last_guesses[counter_num].append([])
                followup_trees[counter_num].append([])
                for ngram_size in range(ngramSizes):
                    records[counter_num][strategy_num].append([])
                    wins[counter_num][strategy_num].append(CompoundGangCounter())
                    wins[counter_num][strategy_num][ngram_size].setDefaultWeights()
                    last_guesses[counter_num][strategy_num].append([])
                    followup_trees[counter_num][strategy_num].append(FollowupTree())
                    for a in [0,2,3]:
                        records[counter_num][strategy_num][ngram_size].append([])
                        last_guesses[counter_num][strategy_num][ngram_size].append(0)

    global my_last_action
    token_list.append(str(observation.lastOpponentAction) + str(my_last_action))
    tree = update_ngram_tree(token_list)

    highestScores = []
    highestScorePlays = []
    highestScoresFollowup = []
    highestScorePlaysFollowup = []
    for i in range(NUM_SCHEMES):
        highestScoresFollowup.append(0)
        highestScorePlaysFollowup.append(-1)
        highestScores.append(0)
        highestScorePlays.append(-1)

    totalScores = [0.0,0.0,0.0]
    totalFollowupScores = [0.0,0.0,0.0]
    for ngramLength in range(ngramSizes):
        for i in range(len(strategyList)):
            for counter in [0, 1, 2]:
                suggestion, scores, followupScores = evaluate_strategy(strategyList, i, records, wins, last_guesses, tree,
                                                                               token_list, ngramLength, counter)

                for scoreIndex in range(NUM_SCHEMES):
                    score = scores[scoreIndex]
                    followupScore = followupScores[scoreIndex]
                    totalScores[int(suggestion)] += score
                    totalFollowupScores[int(suggestion)] += followupScores[scoreIndex]
                    if score > highestScores[scoreIndex]:
                        highestScores[scoreIndex] = score
                        highestScorePlays[scoreIndex] = suggestion
                    if followupScore > highestScoresFollowup[scoreIndex]:
                        highestScoresFollowup[scoreIndex] = followupScore
                        highestScorePlaysFollowup[scoreIndex] = suggestion

    for i in range(NUM_SCHEMES):
        if my_last_actions[i] == beats[observation.lastOpponentAction]:
            score_records[i].count(1)
        else:
            score_records[i].count(0)
        if my_last_actions_followup[i] == beats[observation.lastOpponentAction]:
            score_records_followup[i].count(1)
        else:
            score_records_followup[i].count(0)

    for i in range(NUM_SCHEMES_FINAL):
        if final_actions[i] == beats[observation.lastOpponentAction]:
            final_records[i] += 1
            final_records_counter[i].count(1)
        else:
            final_records_counter[i].count(0)

    for i in range(NUM_SCHEMES_FINAL):
        if final_actions_followup[i] == beats[observation.lastOpponentAction]:
            final_records_followup[i] += 1
            final_records_counter_followup[i].count(1)
        else:
            final_records_counter_followup[i].count(0)
        print('c' + str(i) + ': ' + str(final_records[i]) + ' --- ' + 'f' + str(i) + ': ' + str(final_records_followup[i]))

    finalRecordsTotal = []
    finalRecordsTotalFollowup = []
    for i in range(NUM_SCHEMES_FINAL):
        finalRecordsTotal.append(final_records_counter[i].getWeightedVal())
        finalRecordsTotalFollowup.append(final_records_counter_followup[i].getWeightedVal())

    finalScoreIndex = finalRecordsTotal.index(max(finalRecordsTotal))
    finalScoreIndexFollowup = finalRecordsTotalFollowup.index(max(finalRecordsTotalFollowup))


    for scoreIndex in range(NUM_SCHEMES):
        if highestScores[scoreIndex] > 0:
            my_last_actions[scoreIndex] = highestScorePlays[scoreIndex]
        else:
            my_last_actions[scoreIndex] = randomPlay()
        if highestScoresFollowup[scoreIndex] > 0:
            my_last_actions_followup[scoreIndex] = highestScorePlaysFollowup[scoreIndex]
        else:
            my_last_actions_followup[scoreIndex] = randomPlay()

    finalScores = []
    finalScoresFollowup = []
    bestPlays = []
    bestScores = []
    bestPlaysFollowup = []
    bestScoresFollowup = []
    for j in range(NUM_SCHEMES_FINAL):
        bestPlays.append(-1)
        bestScores.append(0)
        bestPlaysFollowup.append(-1)
        bestScoresFollowup.append(0)
    for i in range(NUM_SCHEMES):
        finalScores.append(score_records[i].getWeightedVals())
        finalScoresFollowup.append(score_records_followup[i].getWeightedVals())
        for j in range(NUM_SCHEMES_FINAL):
            if finalScores[i][j] > bestScores[j]:
                bestScores[j] = finalScores[i][j]
                bestPlays[j] = my_last_actions[i]
            if finalScoresFollowup[i][j] > bestScoresFollowup[j]:
                bestScoresFollowup[j] = finalScoresFollowup[i][j]
                bestPlaysFollowup[j] = my_last_actions_followup[i]
    for scoreIndex in range(NUM_SCHEMES_FINAL):
        final_actions[scoreIndex] = bestPlays[scoreIndex]
        final_actions_followup[scoreIndex] = bestPlaysFollowup[scoreIndex]

    print('max normal: ' + str(max(finalRecordsTotal)) + ', max followup: ' + str(max(finalRecordsTotalFollowup)))
    if max(finalRecordsTotal) > max(finalRecordsTotalFollowup):
        my_last_action = final_actions[finalScoreIndex]
        print('Using RECORD index ' + str(finalScoreIndex))
    else:
        my_last_action = final_actions_followup[finalScoreIndexFollowup]
        print('Using FOLLOWUP index ' + str(finalScoreIndexFollowup))

    if max(max(finalRecordsTotal), max(finalRecordsTotalFollowup)) < 0.38:
        my_last_action = randomPlay()
    if my_last_action == -1:
        my_last_action = randomPlay()
    return my_last_action


def counting_agent(observation, configuration):
    global my_last_action, tally, frame
    if frame > 1 and observation.lastOpponentAction == beats[my_last_action]:
        tally -= 1
    elif frame > 1 and my_last_action == beats[observation.lastOpponentAction]:
        tally += 1
    if observation.step < 10:
        my_last_action = randomPlay()
    else:
        my_last_action = evaluate_records(observation, configuration, False)
    frame += 1
    print('tally: ', tally)
    return my_last_action

Credit goes to @chankhavu's [RPS Dojo](https://www.kaggle.com/chankhavu/rps-dojo) for this code.

In [ ]:
from datetime import datetime
import kaggle_environments

def get_result(team):
    start = datetime.now()
    outcomes = kaggle_environments.evaluate(
        'rps', ['submission.py', team], num_episodes=5)
    won, lost, tie, avg_score = 0, 0, 0, 0.
    for outcome in outcomes:
        score = outcome[0]
        if score > 0: won += 1
        elif score < 0: lost += 1
        else: tie += 1
        avg_score += score
    elapsed = datetime.now() - start
    return won, lost, tie, elapsed, float(avg_score) / 5.0
    
won, lost, tie, elapsed, score = get_result('../input/rps-dojo/black_belt/iocane_powder.py')
print('Counting Agent vs. Iocaine: ' + str(won) + ' / ' + str(lost) + ' / ' + str(tie) + ' // ' + str(score))
